# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_csv_file = "output_data_file"

weather_df = pd.read_csv(weather_csv_file)

weather_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Vaini,-21.2000,-21.2000,74.05,72,100,12.62,TO,1658888792
1,Rikitea,-23.1203,-23.1203,74.73,74,78,8.55,PF,1658888789
2,Busselton,-33.6500,-33.6500,55.35,73,100,8.37,AU,1658888782
3,Guerrero Negro,27.9769,27.9769,70.88,77,8,13.73,MX,1658888858
4,Longyearbyen,78.2186,78.2186,48.04,76,75,3.44,SJ,1658888828


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Convert Humidity to float and store
#  handle NaN values
weather_df = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = weather_df[["Lat", "Lng"]].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Narrow down the DataFrame to find your ideal weather condition.
filtered_weather_df = weather_df

# Drop any rows that don't contain all three conditions. Want to be sure the weather is ideal.

# A max temperature lower than 80 degrees but higher than 70.
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Max Temp"] < 80) & (filtered_weather_df["Max Temp"] > 70)]

# Wind speed less than 10 mph.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind Speed"] < 10]

# Zero cloudiness.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Cloudiness"] == 0]

# Drop any rows with null values
filtered_weather_df = filtered_weather_df.dropna()

filtered_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
77,Akdepe,42.0551,42.0551,75.29,35,0,1.10,TM,1658888982
125,Rio Brilhante,-21.8019,-21.8019,70.20,44,0,5.19,BR,1658888713
154,Qurayyat,31.3318,31.3318,71.78,40,0,9.22,SA,1658888996
158,Kiryat Gat,31.6100,31.6100,74.39,64,0,2.15,IL,1658888997
193,Bethel,41.3712,41.3712,71.65,74,0,3.44,US,1658888806
205,Carbonia,39.1672,39.1672,77.02,79,0,8.43,IT,1658889008
215,Poum,-20.2333,-20.2333,74.91,71,0,9.51,NC,1658889010
217,Kamárai,38.3000,38.3000,77.59,58,0,2.13,GR,1658889010
248,Fayaoué,-20.6500,-20.6500,73.72,67,0,7.61,NC,1658889016
278,Zhezkazgan,47.8043,47.8043,73.38,49,0,8.95,KZ,1658889023


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = filtered_weather_df

# params dictionary to update each iteration
params = {
    "radius": 8000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 77: Akdepe.
Closest hotel is Central Hotel.
------------
Retrieving Results for Index 125: Rio Brilhante.
Missing field/result... skipping.
------------
Retrieving Results for Index 154: Qurayyat.
Closest hotel is منزل الحاج عبدالوهاب العشماوى.
------------
Retrieving Results for Index 158: Kiryat Gat.
Missing field/result... skipping.
------------
Retrieving Results for Index 193: Bethel.
Closest hotel is Paluri Hotel.
------------
Retrieving Results for Index 205: Carbonia.
Closest hotel is Hozat Ogretmenevi Aksam Sanat Okulu.
------------
Retrieving Results for Index 215: Poum.
Missing field/result... skipping.
------------
Retrieving Results for Index 217: Kamárai.
Closest hotel is Yesilyurt Ogretmenevi.
------------
Retrieving Results for Index 248: Fayaoué.
Missing field/result... skipping.
------------
Retrieving Results for Index 278: Zhezkazgan.
Missing field/result... skipping.
------------
Retrieving Results for Index 283: Partenit.
Missing field

In [6]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
77,Akdepe,42.0551,42.0551,75.29,35,0,1.10,TM,1658888982,Central Hotel
125,Rio Brilhante,-21.8019,-21.8019,70.20,44,0,5.19,BR,1658888713,NaN
154,Qurayyat,31.3318,31.3318,71.78,40,0,9.22,SA,1658888996,منزل الحاج عبدالوهاب العشماوى
158,Kiryat Gat,31.6100,31.6100,74.39,64,0,2.15,IL,1658888997,NaN
193,Bethel,41.3712,41.3712,71.65,74,0,3.44,US,1658888806,Paluri Hotel
205,Carbonia,39.1672,39.1672,77.02,79,0,8.43,IT,1658889008,Hozat Ogretmenevi Aksam Sanat Okulu
215,Poum,-20.2333,-20.2333,74.91,71,0,9.51,NC,1658889010,NaN
217,Kamárai,38.3000,38.3000,77.59,58,0,2.13,GR,1658889010,Yesilyurt Ogretmenevi
248,Fayaoué,-20.6500,-20.6500,73.72,67,0,7.61,NC,1658889016,NaN
278,Zhezkazgan,47.8043,47.8043,73.38,49,0,8.95,KZ,1658889023,NaN


In [7]:
# Assign the marker layer to a variable
locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map

fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))